# Protected Area Boundary Change

This notebook outlines the methodology used to measure at protected area boundaries via GEE. 

The notebook queries MODIS imagery and returns the gradient values of relevant bands as well as the vegetation indices NDVI and EVI. This code expects an annual time span and returns a geoTIFF for each band in each park for that year.

In [1]:
from utils import *
from config import *
import pandas as pd
from datetime import datetime

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='dse-staff')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


## Class Definitions

In [3]:
def main(wdpaid, year, show_map=False, band_name=None):
    """Main function to process protected area boundary analysis"""
    # Initialize classes
    geo_ops = GeometryOperations()
    img_ops = ImageOperations()
    stats_ops = StatsOperations()
    viz = Visualization()
    feature_processor = FeatureProcessor(geo_ops, img_ops, stats_ops)
    exporter = ExportResults()

    # Load and process protected area geometry
    pa = load_local_data(wdpaid)
    pa_geometry = pa.geometry()
    aoi = geo_ops.buffer_polygon(pa_geometry)
    aoi = geo_ops.mask_water(aoi)
    aoi_with_biome = geo_ops.get_biome(aoi)

    # Process imagery and add indices
    modis_ic = img_ops.modis.filter(img_ops.filter_for_year(aoi, year))
    band_names = modis_ic.first().bandNames()
    composite = modis_ic.reduce(ee.Reducer.median()).rename(band_names).clip(aoi)
    image = img_ops.add_indices_to_image(composite)

    # Process features and collect statistics
    feature_info = feature_processor.collect_feature_info(pa, aoi_with_biome)
    computed_stats = feature_processor.process_all_bands(image, pa_geometry, aoi)
    all_stats = feature_processor.compile_statistics(feature_info, computed_stats, year)
    
    # Save results
    df = pd.DataFrame(all_stats)
    exporter.save_df_to_gcs(df, 'dse-staff', wdpaid, year)

    # Visualization
    if show_map:
        band_stats = next(cs for cs in computed_stats if cs["band_name"] == band_name)
        Map = viz.create_map(pa_geometry, band_stats['buffer_pixels'], band_stats['boundary_pixels'])

    return print("Analysis complete for WDPA ID:", wdpaid, "for the year:", year)

In [4]:
#Map = main("916", 2010) #show_map=True, band_name="EVI")
#Map

In [5]:
import concurrent.futures

def run_main_for_years(wdpaid, start_year, n_years, show_map=False, band_name=None, max_workers=4):
    """
    Runs main() in parallel for a sequence of years.
    Prints completion messages as each run finishes.
    """
    years = [start_year + i for i in range(n_years)]
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_year = {
            executor.submit(main, wdpaid, year, show_map, band_name): year for year in years
        }
        for future in concurrent.futures.as_completed(future_to_year):
            year = future_to_year[future]
            try:
                future.result()  # main() already prints its own completion message
            except Exception as exc:
                print(f"Year {year} generated an exception: {exc}")

In [6]:
run_main_for_years("916", 2001, 3, max_workers=4)

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD09A1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD09A1

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for JRC/GSW1_0/GlobalSurfaceWater! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_0_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" erro

Uploaded to: gs://dse-staff/protected_areas/tables/_916_2003_20250624_235320.csv
Analysis complete for WDPA ID: 916 for the year: 2003


/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Uploaded to: gs://dse-staff/protected_areas/tables/_916_2001_20250624_235320.csv
Analysis complete for WDPA ID: 916 for the year: 2001


/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Uploaded to: gs://dse-staff/protected_areas/tables/_916_2002_20250624_235320.csv
Analysis complete for WDPA ID: 916 for the year: 2002


-add write out for each step, with identifier for each park
-ray to run in parallel in python
-use glance to check usage